In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import uuid

num_records = 100

data = []

for _ in range(num_records):
    timestamp = datetime.now() - timedelta(minutes=np.random.randint(0, 1440))  
    block_number = np.random.randint(1000000, 2000000)
    
    record = {
        "transaction_hash": f"0x{uuid.uuid4().hex}",  
        "contract_address": f"0x{uuid.uuid4().hex[:40]}", 
        "block_number": block_number,
        "event_type": "VitalSignsRecorded", 
        "timestamp": timestamp.isoformat(),
        "patient_id": f"PAT{np.random.randint(100, 999)}",
        "heart_rate": np.random.randint(60, 100),
        "blood_pressure": f"{np.random.randint(100, 140)}/{np.random.randint(60, 90)}",
        "oxygen_level": np.random.randint(95, 100),
        "body_temp": round(np.random.uniform(36.0, 38.0), 1)
    }
    data.append(record)


df = pd.DataFrame(data)

df.to_csv("smart_contract_health_data.csv", index=False)
df.to_json("smart_contract_health_data.json", orient="records")

df.head()


In [ ]:
from datetime import datetime
from typing import List


class VitalSigns:
    def __init__(self, timestamp, patient_id, heart_rate, blood_pressure, oxygen_level, body_temp):
        self.timestamp = timestamp
        self.patient_id = patient_id
        self.heart_rate = heart_rate
        self.blood_pressure = blood_pressure
        self.oxygen_level = oxygen_level
        self.body_temp = body_temp  

    def to_dict(self):
        return {
            "timestamp": self.timestamp.isoformat(),
            "patient_id": self.patient_id,
            "heart_rate": self.heart_rate,
            "blood_pressure": self.blood_pressure,
            "oxygen_level": self.oxygen_level,
            "body_temp": self.body_temp / 10  
        }


class HealthDataStorage:
    MAX_ENTRIES = 100

    def __init__(self, owner):
        self.owner = owner
        self.health_records: List[VitalSigns] = []

    def store_vital_signs(self, caller, patient_id, heart_rate, blood_pressure, oxygen_level, body_temp):
        if caller != self.owner:
            raise PermissionError("Not authorized")

        if len(self.health_records) >= self.MAX_ENTRIES:
            raise OverflowError("Storage limit reached")

        timestamp = datetime.now()
        record = VitalSigns(timestamp, patient_id, heart_rate, blood_pressure, oxygen_level, body_temp)
        self.health_records.append(record)

        print(f"[Event] VitalSignsRecorded: {record.to_dict()}")

    def get_total_records(self):
        return len(self.health_records)

    def get_record(self, index):
        if index >= len(self.health_records):
            raise IndexError("Index out of bounds")
        return self.health_records[index].to_dict()
